In [2]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from bs4 import NavigableString

In [3]:
class crawlerPtt():
    def __init__(self,url,pageRange, start_page_id):
        # 欲抓取的看板首頁
        self.url = url
        # 儲存每個標題網址
        self.urlList = []
        # 儲存每條留言
#         self.messageList = []
        # 儲存每條留言
#         self.titleList = []
        self.data = pd.DataFrame()

        # 抓取首頁
        self.get_all_href(url=self.url)

        # 往前幾頁 抓取所有標題網址
        for page in range(1, pageRange):
            # 使用 GET 方式下載普通網頁
            r = requests.get(self.url)
            # html.parser 為解析 HTML 文件的模組
            soup = BeautifulSoup(r.text, "html.parser")
            # 抓取按鈕群
            btn = soup.select('h2 > a')
            # 抓取上一頁按鈕的網址
            up_page_href = btn[3]['href']
            # 在網址後加上上一頁的網址
            next_page_url = 'https://www.ptt.cc' + up_page_href
            # 爬取上一頁內容 並更新url成上一頁的網址
            self.url = next_page_url
            self.get_all_href(url=self.url)

        # 透過標題網址self.urlList抓取留言 並儲存到self.messageList最後再存下來 (邊抓邊存會漏掉很多留言)
        self.crawlerMessage()

    # 抓取標題網址
    def get_all_href(self,url):
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        # 抓取文章標題
        results = soup.select("div.title")
        # 尋訪每個標題 取得網址
        for item in results:
            a_item = item.select_one("a")
            # 標題名稱
            title = item.text
            if a_item:
                
                #所要儲存的網站網址
                url = 'https://www.ptt.cc' + a_item.get('href')
                print(url)
                # 儲存網址至佇列
                self.urlList.append(url)
                
    def getResult(self):
        return self.data
    

    # 從標題網址分析留言
    def crawlerMessage(self):
        # parsing
        all_posts = []
        for link in self.urlList:
            try:
                response = requests.get(link).text
                soup = BeautifulSoup(response, 'html.parser')
            except:
                # ignore this link
                continue

            # meta info
            metas = soup.find_all(class_ = 'article-meta-value')
            if(len(metas) == 0):
                continue
            title = metas[-2].text
            author = metas[0].text
            time = metas[-1].text

            # content
            content = ''
            for text in soup.find(id='main-content'):
                if isinstance(text, NavigableString):
                    content += text.strip()

            # comments(a list of list, each includes push_tag and push_content)
            comments = []
            for div in soup.find_all(class_ = 'push'):
                push_tag, push_content = '', ''
                try:
                    # push_tag's type is <class 'bs4.element.ResultSet'> (?
                    push_tag = div.find('span', 'push-tag').contents[0]
                    push_content = div.find('span', 'push-content').text
                    push_tag = push_tag.strip()
                    push_content = push_content[2:]
                except:
                    pass
                comments.append([push_tag, push_content])

            # 將上面的每一篇文章的資訊存成一個 dictionary，並把這些 dicts 存進 all_posts (list)
            all_posts.append({'title':title, 'content':content, 'author':author, 'link':link, 'postTime':time, 'comments':comments})
        self.data = pd.DataFrame(all_posts)

In [7]:
r = requests.get("https://www.dcard.tw/f/relationship")
# html.parser 為解析 HTML 文件的模組
soup = BeautifulSoup(r.text, "html.parser")
# 抓取按鈕群
btn = soup.select('h2 > a')
# 抓取上一頁按鈕的網址
print(btn[0])

<a class="tgn9uw-3 iuyCWN" href="/f/relationship/p/234885552"><span>發現男友秘密不知道該不該分手...</span></a>


In [ ]:
我男友跟我姐聊的很開心，還問她生日是什麼時候，直到剛剛我看到我姐的訊息（她都會讓我看），有很多則訊息 其中一則上面寫：其實是想見妳，我看到直接一直發抖 也哭不出來，他跟我姐聊的比跟我聊的還多